<a href="https://colab.research.google.com/github/nghiaphan177/Truyvanttdpt/blob/main/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir data

In [3]:
cd data

/content/data


In [4]:
import requests
import bs4
import datetime
import re
import os

In [5]:
def sendRequest(url):
  while True:
    try:
      rp = requests.get(url, timeout=1)
      break
    except:
      continue
  bs4Rp = bs4.BeautifulSoup(rp.text,"html.parser")
  return bs4Rp

In [6]:
def filterPtag(pTag):
  if not pTag.attrs.get('class', False):
    return False
  if 'Image' in pTag.attrs['class']:
    return False
  if pTag.attrs.get('style', False):
    return False
  if pTag.attrs.get('align', False):
    return False
  return True


In [7]:
def isText(htmlArticle):
  tag = htmlArticle.find('meta', {'name' : 'its_type'})
  if not tag:
    return False
  if tag.attrs['content'] == 'text':
    return True
  return False

In [8]:
def getContent(urlPage):
  htmlArticle = sendRequest(urlPage)
  contentPage = ''
  if not isText(htmlArticle):
    return contentPage
  allPtags = htmlArticle.find_all('p')[:-1]
  usefulPtags = filter(filterPtag, allPtags)
  for pTag in usefulPtags:
    text = pTag.text
    contentPage = contentPage + '\n' + text 
  return contentPage

In [9]:
def crawlData(rootUrl, folder2Save, amount):
  # get the current path
  currPath = os.getcwd()
  # check if folder2Save is exist else create
  dir = os.path.join(currPath, folder2Save)
  if not os.path.exists(dir):
    os.mkdir(dir)

  url = rootUrl
  
  number = 0
  i = 0
  flag = True
  while flag:
    # Find all the links of the a category page 
    bs4Rp = sendRequest(url)
    articleLinks = set(re.findall(r'href=\"(.+\.html)', str(bs4Rp)))
    # Go to the article page and get the content
    for articleLink in articleLinks:
      contentPage = getContent(articleLink)
      if contentPage == '':
        continue
      number += 1
      nameFile = f'data-{number}.txt'
      pathFileTxt = os.path.join(dir, nameFile)
      with open(pathFileTxt, 'a+', encoding='utf-8') as f:
        f.write(contentPage)
      if number == amount:
        flag = False
        break
    # set up url to  the next page 
    i += 1
    url = rootUrl + f"-p{i}"

In [10]:
rootUrl = 'https://vnexpress.net/phap-luat'
folder2Save = 'phap-luat'
amount = 200
crawlData(rootUrl, folder2Save, amount)

In [11]:
rootUrl = 'https://vnexpress.net/kinh-doanh'
folder2Save = 'kinh-doanh'
amount = 200
crawlData(rootUrl, folder2Save, amount)

In [12]:
rootUrl = 'https://vnexpress.net/giai-tri'
folder2Save = 'giai-tri'
amount = 200
crawlData(rootUrl, folder2Save, amount)

**Lưu data thành tập tin csv có 2 cột là content và category**

In [13]:
import pandas as pd

In [14]:
dataPath = '/content/data'
df = pd.DataFrame()
for dir in os.listdir(dataPath):
  if dir == '.ipynb_checkpoints':
    continue
  pathDir = os.path.join(dataPath, dir)
  for txtFile in os.listdir(pathDir):
    pathTxtFile = os.path.join(pathDir, txtFile)
    with open(pathTxtFile, 'r', encoding='utf-8') as f:
      text = f.read()
    df  = df.append([[text, dir]], ignore_index=True)
df = df.rename(columns={0 : 'content', 1 : 'category'})

In [25]:
df.to_csv('data.csv')